### Load tensorflow

In [5]:
import tensorflow as tf
tf.compat.v1.reset_default_graph()
tf.compat.v1.set_random_seed(42)
#tf.reset_default_graph()
#tf.set_random_seed(42)

### Read the data
<font size="2">Data for this exercise can be downloaded from http://www.manythings.org/anki/</font>

In [6]:
!ls

hin-eng.zip  sample_data


In [7]:
#You can use wget to download the file directly
!wget http://www.manythings.org/anki/hin-eng.zip
#!wget http://www.manythings.org/anki/spa-eng.zip 

--2022-03-03 02:47:23--  http://www.manythings.org/anki/hin-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.92.44, 172.67.186.54, 2606:4700:3033::ac43:ba36, ...
Connecting to www.manythings.org (www.manythings.org)|104.21.92.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133034 (130K) [application/zip]
Saving to: ‘hin-eng.zip.1’

hin-eng.zip.1       100%[===================>] 129.92K  --.-KB/s    in 0.09s   

2022-03-03 02:47:23 (1.43 MB/s) - ‘hin-eng.zip.1’ saved [133034/133034]



In [8]:
import zipfile
import io

#Read the zip file
zf = zipfile.ZipFile('hin-eng.zip', 'r')

#Extract data from zip file
data = ''
with zf.open('hin.txt') as readfile:
  for line in io.TextIOWrapper(readfile, 'utf-8'):
    data += line

In [9]:
data[400:500]

' (France) Attribution: tatoeba.org #631038 (Shishir) & #6179121 (fastrizwaan)\nJump.\tकूदो.\tCC-BY 2.0 '

In [10]:
data = data[0:3300000]


### Extract Source and Target Language pairs

In [11]:
#Split by newline character
data =  data.split('\n')

#Show some Data
data[100:105]

["What's this?\tयह क्या है?\tCC-BY 2.0 (France) Attribution: tatoeba.org #413821 (CK) & #443158 (minshirui)",
 'Are you sick?\tक्या तुम बीमार हो?\tCC-BY 2.0 (France) Attribution: tatoeba.org #434252 (lukaszpp) & #518699 (minshirui)',
 'Bring him in.\tउसको अंदर ले आओ।\tCC-BY 2.0 (France) Attribution: tatoeba.org #307895 (CK) & #475932 (minshirui)',
 'Come with us.\tहमारे साथ आओ।\tCC-BY 2.0 (France) Attribution: tatoeba.org #433696 (CK) & #485546 (minshirui)',
 'Happy Easter!\tएसटर मुबारक हो!\tCC-BY 2.0 (France) Attribution: tatoeba.org #66762 (papabear) & #3189572 (pranjal710)']

In [12]:
len(data)

2935

### Separate Source and Target pairs

In [13]:
encoder_text = [] #Initialize Source language list
decoder_text = [] #Initialize Target language list

#Iterate over data
for line in data:
    try:
        in_txt, out_txt,_ = line.split('\t')
        encoder_text.append(in_txt)
        
        # Add tab '<start>' as 'start sequence in target
        # And '<end>' as End
        decoder_text.append('<start> ' + out_txt + ' <end>')
    except:
        pass #ignore data which goes into error        

### Separate Source and Target pairs..

In [14]:
print(encoder_text[100:160],end = "")

["What's this?", 'Are you sick?', 'Bring him in.', 'Come with us.', 'Happy Easter!', 'Has Tom left?', 'I am at home.', "I can't move.", "I don't know.", "I don't know.", 'I have a car.', 'I have a dog.', 'I understand.', "I'm a doctor.", 'It is a book.', "It's snowing.", "It's too big.", 'Please leave.', 'Unbelievable!', 'We are happy.', 'What is this?', 'Are you tired?', 'Can you drive?', 'Do you get it?', "Don't get fat.", "Don't give in.", 'Drink it down.', 'Everyone dies.', 'Flowers bloom.', 'I am who I am.', 'I know things.', 'I like donuts.', "I'll take him.", "I'm tired now.", "I'm very busy.", 'Is that a cat?', "It's for free.", "It's for free.", 'Let me try it.', 'Let me try it.', 'Let me try it.', "Let's do that.", 'Make it quick.', 'May I come in?', 'Open the door.', 'Open the door.', 'Please get in.', 'Read it again.', 'Read it aloud.', 'She bent down.', 'Some fish fly.', 'This is a map.', 'Tom is my son.', "We're in town.", 'Were you shot?', 'What about us?', 'Can I help y

In [15]:
decoder_text[100:105]

['<start> यह क्या है? <end>',
 '<start> क्या तुम बीमार हो? <end>',
 '<start> उसको अंदर ले आओ। <end>',
 '<start> हमारे साथ आओ। <end>',
 '<start> एसटर मुबारक हो! <end>']

### Tokenize Source language sentences

In [16]:
#Tokenizer for source language
encoder_t = tf.keras.preprocessing.text.Tokenizer()
encoder_t.fit_on_texts(encoder_text) #Fit it on Source sentences
encoder_seq = encoder_t.texts_to_sequences(encoder_text) #Convert sentences to numbers 
encoder_seq[100:105] #Display some converted sentences

[[173, 15], [20, 4, 402], [403, 29, 9], [50, 37, 81], [153, 1311]]

In [17]:
#Maximum length of sentence
max_encoder_seq_length = max([len(txt) for txt in encoder_seq])
print('Maximum sentence length for Source language: ', max_encoder_seq_length)

#Source language Vocablury
encoder_vocab_size = len(encoder_t.word_index)
print('Source language vocablury size: ', encoder_vocab_size)

Maximum sentence length for Source language:  22
Source language vocablury size:  2412


### Tokenize Target language sentences

In [18]:
#Tokenizer for target language, filters should not <start> and <end>
#remove < and > used in Target language sequences
decoder_t = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
decoder_t.fit_on_texts(decoder_text) #Fit it on target sentences
decoder_seq = decoder_t.texts_to_sequences(decoder_text) #Convert sentences to numbers 

In [19]:
#Maximum length of sentence
max_decoder_seq_length = max([len(txt) for txt in decoder_seq])
print('Maximum sentence length for Target language: ', max_decoder_seq_length)

#Target language Vocablury
decoder_vocab_size = len(decoder_t.word_index)
print('Target language vocablury size: ', decoder_vocab_size)

Maximum sentence length for Target language:  27
Target language vocablury size:  3036


### Compare different sentences length

In [20]:
#Source Language sentences
print('Length for sentence number 100: ', len(encoder_seq[100]))
print('Length for sentence number 2000: ', len(encoder_seq[2000]))

Length for sentence number 100:  2
Length for sentence number 2000:  7


In [21]:
#Target Language sentences
print('Length for sentence number 100: ', len(decoder_seq[100]))
print('Length for sentence number 2000: ', len(decoder_seq[2000]))

Length for sentence number 100:  5
Length for sentence number 2000:  10


### How do we make it same?

### Padding the sentences

In [22]:
#Source sentences
encoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(encoder_seq, 
                                                                   maxlen=max_encoder_seq_length, #22
                                                                   padding='pre')

#Target Sentences
decoder_input_data = tf.keras.preprocessing.sequence.pad_sequences(decoder_seq, 
                                                                   maxlen=max_decoder_seq_length, #27
                                                                   padding='post')

In [23]:
print('Source data shape: ', encoder_input_data.shape)
print('Target data shape: ', decoder_input_data.shape)

Source data shape:  (2934, 22)
Target data shape:  (2934, 27)


#### Integer to Word converter for Decoder data

In [24]:
int_to_word_decoder = dict((i,c) for c, i in decoder_t.word_index.items())

In [25]:
int_to_word_decoder[15]

'हैं।'

### Building Decoder Output

In [26]:
import numpy as np

#Initialize array
decoder_target_data = np.zeros((decoder_input_data.shape[0], decoder_input_data.shape[1]))

#Shift Target output by one word
for i in range(decoder_input_data.shape[0]):
    for j in range(1,decoder_input_data.shape[1]):
        decoder_target_data[i][j-1] = decoder_input_data[i][j]

#### Convert target data in one hot vector

In [27]:
#Initialize one hot encoding array
decoder_target_one_hot = np.zeros((decoder_input_data.shape[0], #number of sentences
                                   decoder_input_data.shape[1], #Number of words in each sentence
                                   len(decoder_t.word_index)+1)) #Vocab size + 1

In [28]:
#Build one hot encoded array
for i in range(decoder_target_data.shape[0]):
    for j in range(decoder_target_data.shape[1]):
        decoder_target_one_hot[i][j] = tf.keras.utils.to_categorical(decoder_target_data[i][j],
                                                                     num_classes=len(decoder_t.word_index)+1)    

In [29]:
decoder_target_one_hot.shape

(2934, 27, 3037)

### Building the Training Model

In [30]:
#Define config parameters
encoder_embedding_size = 50
decoder_embedding_size = 50
rnn_units = 256

#### Build Encoder

In [31]:
#Input Layer
encoder_inputs = tf.keras.layers.Input(shape=(None,))

#Embedding layer
encoder_embedding = tf.keras.layers.Embedding(encoder_vocab_size+1, encoder_embedding_size)

#Get embedding layer output by feeding inputs
encoder_embedding_output = encoder_embedding(encoder_inputs)

#---Following code has been commented out for Attention-------
#LSTM Layer and its output
#x, state_h, state_c = tf.keras.layers.LSTM(rnn_units,return_state=True)(encoder_embedding_output)

#Build a list to feed Decoder
#encoder_states = [state_h, state_c]

#### Build Encoder - Get all hidden states

In [32]:
#Create LSTM Layer and get All hidden states, last hidden and cell state
encoder_lstm = tf.keras.layers.LSTM(rnn_units,return_state=True, return_sequences=True)

#Get 3 outputs of LSTM Layer
encoder_all_h_states, state_h, state_c = encoder_lstm(encoder_embedding_output)

#Build a list to feed Decoder
encoder_states = [state_h, state_c]

#### Build Decoder

In [33]:
#Decode input - padded Target sentences
decoder_inputs = tf.keras.layers.Input(shape=(None,))

#Decoder Embedding layer
decoder_embedding = tf.keras.layers.Embedding(decoder_vocab_size + 1, decoder_embedding_size)

#Embedding layer output
decoder_embedding_output = decoder_embedding(decoder_inputs)

#Decoder RNN
decoder_rnn = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)

#Decoder RNN Output, State initialization from Encoder states
#Output will be all hidden sequences, last 'h' state and last 'c' state
decoder_all_h_states,_,_ = decoder_rnn(decoder_embedding_output, 
                                       initial_state=encoder_states)

#---Following code has been commented out for Attention-------
#Output Layer
#decoder_dense = tf.keras.layers.Dense(decoder_vocab_size + 1, activation='softmax')

#Output of Dense layer
#decoder_outputs = decoder_dense(x)

#### Build Decoder...Alignment Matrix

In [34]:
#1. Dot Product between Decoder_all_h_states and encoder_all_h_states
#2. Apply softmax to get Alignment matrix

#Dimensions details
#decoder_all_states = batch_size x max_decoder_length x rnn_units
#encoder_all_states = batch_size x max_encoder_length x rnn_units
#score = batch_size x max_decoder_length x max_encoder_length
#alignment matrix = batch_size x max_decoder_length x max_encoder_length

score = tf.keras.layers.dot([decoder_all_h_states, encoder_all_h_states], axes=2)
alignment_matrix = tf.keras.layers.Activation('softmax')(score)

#Try general and concat approaches to alignment matrix

In [35]:
alignment_matrix

<KerasTensor: shape=(None, None, None) dtype=float32 (created by layer 'activation')>

#### Build Decoder...Context Vector

In [36]:
#Weighted sum of multiplication of Alignment matrix and encoder states
#Dimension of context_vector =  batch_size x max_decoder_length x rnn_units

context_vector = tf.keras.layers.dot([alignment_matrix, encoder_all_h_states], axes=[2,1])

In [37]:
context_vector

<KerasTensor: shape=(None, None, 256) dtype=float32 (created by layer 'dot_1')>

#### Build Decoder...Attention Vector

In [38]:
#Concatenate context vector and decoder_all_h_states
#context_decoder_hidden = batch_size x max_decoder_length x rnn_units
#attention_vector = batch_size x max_decoder_length x 128

context_decoder_hidden = tf.keras.layers.concatenate([context_vector, 
                                                      decoder_all_h_states])

attention_dense_layer = tf.keras.layers.Dense(128, use_bias=False, 
                                              activation='tanh')

attention_vector = attention_dense_layer(context_decoder_hidden)

In [39]:
attention_vector

<KerasTensor: shape=(None, None, 128) dtype=float32 (created by layer 'dense')>

#### Build Decoder...Output layer

In [40]:
#Output layer
decoder_dense = tf.keras.layers.Dense(decoder_vocab_size + 1, activation='softmax')

#With attention input will be attention_vector and not decoder_all_h_states
decoder_outputs = decoder_dense(attention_vector)

### Build Model using both Encoder and Decoder

In [41]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], #2 Inputs to the model
                              decoder_outputs) #Output of the model

In [42]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Train the model

In [43]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_one_hot,
          batch_size=64,
          epochs=25,
          validation_split=0.2)

Epoch 1/25
37/37 [==============================] - 22s 476ms/step - loss: 4.1327 - accuracy: 0.7180 - val_loss: 3.2997 - val_accuracy: 0.5917
Epoch 2/25
37/37 [==============================] - 16s 444ms/step - loss: 1.9719 - accuracy: 0.7381 - val_loss: 3.0143 - val_accuracy: 0.5917
Epoch 3/25
37/37 [==============================] - 16s 444ms/step - loss: 1.7480 - accuracy: 0.7381 - val_loss: 2.9737 - val_accuracy: 0.5917
Epoch 4/25
37/37 [==============================] - 19s 507ms/step - loss: 1.6582 - accuracy: 0.7396 - val_loss: 2.8875 - val_accuracy: 0.5949
Epoch 5/25
37/37 [==============================] - 16s 443ms/step - loss: 1.5576 - accuracy: 0.7522 - val_loss: 2.6397 - val_accuracy: 0.6108
Epoch 6/25
37/37 [==============================] - 16s 445ms/step - loss: 1.4596 - accuracy: 0.7754 - val_loss: 2.5831 - val_accuracy: 0.6299
Epoch 7/25
37/37 [==============================] - 16s 444ms/step - loss: 1.4031 - accuracy: 0.7816 - val_loss: 2.5358 - val_accuracy: 0.6347

### Save the model for later reuse

In [44]:
model.save('models/seq2seq_training_translation_attention.hd5')

INFO:tensorflow:Assets written to: models/seq2seq_training_translation_attention.hd5/assets


INFO:tensorflow:Assets written to: models/seq2seq_training_translation_attention.hd5/assets


In [45]:
model = tf.keras.models.load_model('models/seq2seq_training_translation_attention.hd5')

# Building Model for Prediction

### Build the Encoder Model to predict Encoder States

In [46]:
encoder_model = tf.keras.models.Model(inputs=encoder_inputs, #Padded input sequences
                                      outputs=[encoder_all_h_states] + #Hidden states at all time steps
                                      encoder_states) #Hidden state and Cell state at last time step

### Build the Decoder Model 
<p/>

<ol><li>Define Input for both 'h' state and 'c' state initialization </li>
    <li><font color="blue">Define Input for all encoder states - Attention Layer </font></li>
<li>Get Decoder RNN outputs along with h and c state</li>
<li><font color="blue">Build Attention Layer</font></li>
<li><font color="blue">Get Decoder Dense layer output using Attention vector</font></li>
    <li><font color="blue">Build Model</font></li></ol>

##### Step 1 - Define Input for both 'h' state and 'c' state initialization

In [47]:
#Hidden state input
decoder_state_input_h = tf.keras.layers.Input(shape=(rnn_units,))

#Cell state input
decoder_state_input_c = tf.keras.layers.Input(shape=(rnn_units,))

#Putting it together
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

##### Step 2 - Define Input encoder states - Attention Layer

In [48]:
encoder_outputs = tf.keras.layers.Input(shape=(max_encoder_seq_length, rnn_units,))

##### Step 3 - Get Decoder RNN outputs along with h and c state

In [49]:
#Get Embedding layer output
x = decoder_embedding(decoder_inputs)

#We will use the layer which we trained earlier
rnn_outputs, state_h, state_c = decoder_rnn(x, initial_state=decoder_states_inputs)

#Why do we need this?
decoder_states = [state_h, state_c]

##### Step 4 - Build Attention Layer

In [50]:
#Alignment score
p_score = tf.keras.layers.dot([rnn_outputs, encoder_outputs], axes=2)

#Perform softmax to get Alignment matrix
p_alignment_matrix = tf.keras.layers.Activation('softmax')(p_score)

#Context Vector
p_context_vector = tf.keras.layers.dot([p_alignment_matrix, encoder_outputs], axes=[2,1])

#Build Attention Vector
# 1. Caoncatenate both context vector and decoder outputs
# 2. Feed it to the Dense layer 
p_context_decoder_hidden = tf.keras.layers.concatenate([p_context_vector, rnn_outputs])
p_attention_vector = attention_dense_layer(p_context_decoder_hidden)

In [51]:
p_alignment_matrix

<KerasTensor: shape=(None, None, 22) dtype=float32 (created by layer 'activation_1')>

##### Step 5 - Get Decoder Dense layer output

In [52]:
decoder_outputs = decoder_dense(p_attention_vector)

##### Step 6 - Build Decoder Model

In [53]:
#3 Inputs - Word, h/c state and all hidden states from encoder
#3 Outputs - predicted word, h and c state values for next run and alignment matrix for visualization

decoder_model = tf.keras.models.Model([decoder_inputs] +  #Start sequence and then word
                                      decoder_states_inputs + #h and c state value for initialization
                                      [encoder_outputs],  #Encoder all hidden states for Attention layer
                                      [decoder_outputs] + #Model word prediction
                                      decoder_states +   #h and c states for next run
                                      [p_alignment_matrix]) #for Alignment matrix

# Predicting output from Seq2Seq model

##### Build a prediction function

In [54]:
def decode_sentence(input_sequence):
    
    #Get the encoder state values
    encoder_output =  encoder_model.predict(input_sequence)
    decoder_initial_states_value = encoder_output[1:]    
    encoded_seqs = encoder_output[0]
    
    #Build a sequence with '<start>' - starting sequence for Decoder
    target_seq = np.zeros((1,1))    
    target_seq[0][0] = decoder_t.word_index['<start>']
    
    #flag to check if prediction should be stopped
    stop_loop = False
    
    #Initialize predicted sentence
    predicted_sentence = ''
    
    #start the loop
    while not stop_loop:
        
        #Decoder model with 3 inputs
        predicted_outputs, h, c, a_matrix = decoder_model.predict([target_seq] + 
                                                                  decoder_initial_states_value +
                                                                  [encoded_seqs])
        
        #Get the predicted word index with highest probability
        predicted_output = np.argmax(predicted_outputs[0,-1,:])
        
        #Get the predicted word from predicter index
        if (predicted_output == 0):
            predicted_word = ' '
        else:
            predicted_word = int_to_word_decoder[predicted_output]
        
        #Check if prediction should stop
        if(predicted_word == '<end>' or len(predicted_sentence) > max_decoder_seq_length):
            
            stop_loop = True
            continue
                    
        #Updated predicted sentence
        if (len(predicted_sentence) == 0):
            predicted_sentence = predicted_word
        else:
            predicted_sentence = predicted_sentence + ' ' + predicted_word
            
        #Update target_seq to be the predicted word index
        target_seq[0][0] = predicted_output
        
        #Update initial states value for decoder
        decoder_initial_states_value = [h,c]
        
        #print(a_matrix)
    
    return predicted_sentence

##### Call Prediction function on a random sentence

In [55]:
#Generate a random number
start_num = np.random.randint(0, high=len(encoder_text) - 10)

#Predict model output for 5 sentences
for i in range(start_num, start_num + 1):
    input_seq = encoder_input_data[i : i+1]
    #print(input_seq)
    predicted_sentence = decode_sentence(input_seq)
    print('--------')
    print ('Input sentence: ', encoder_text[i])
    print ('Predicted sentence: ', predicted_sentence )

--------
Input sentence:  I want you to keep your promise.
Predicted sentence:  मैं मुझे एक एक बहुत नहीं नहीं


##### Save encoder and decoder model

In [56]:
#Compile models to avoid error
encoder_model.compile(optimizer='adam',loss='categorical_crossentropy')
decoder_model.compile(optimizer='adam',loss='categorical_crossentropy')

#Save the models
encoder_model.save('models/seq2seq_encoder_eng_hin.hd5')  #Encoder model
decoder_model.save('models/seq2seq_decoder_eng_hin.hd5')  #Decoder model

INFO:tensorflow:Assets written to: models/seq2seq_encoder_eng_hin.hd5/assets


INFO:tensorflow:Assets written to: models/seq2seq_encoder_eng_hin.hd5/assets


INFO:tensorflow:Assets written to: models/seq2seq_decoder_eng_hin.hd5/assets


INFO:tensorflow:Assets written to: models/seq2seq_decoder_eng_hin.hd5/assets


##### Save encoder and decoder tokenizers

In [57]:
import pickle

pickle.dump(encoder_t,open('models/encoder_tokenizer_eng','wb'))
pickle.dump(decoder_t,open('models/decoder_tokenizer_hin','wb'))